<a href="https://colab.research.google.com/github/sunkingbms/zendeskUsers/blob/main/Telerivet_Kenya_Error_Log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
import time
import telerivet

def get_failed_messages(API_KEY: str, ken_project_id: str, delay: int = 6) -> list:
    """
    Retrieve logs with a fail status and return an array of all the errors from the current day
    """
    messages = []
    today = datetime.date.today()

    start_of_day = int(datetime.datetime.combine(today, datetime.time.min).timestamp())
    end_of_day = int(datetime.datetime.combine(today, datetime.time.max).timestamp())

    tr = telerivet.API(API_KEY)
    project = tr.initProjectById(ken_project_id)

    try:
        cursor = project.queryMessages(
            direction="outgoing",
            message_type="sms",
            status="failed",
            time_created={"min": start_of_day, "max": end_of_day}
        )

        for message in cursor.all():
            messages.append({
                "status": message.status,
                "error log": message.error_message,
                "from number": message.from_number,
                "to number": message.to_number,
                "time_sent": datetime.datetime.utcfromtimestamp(message.time_sent).strftime('%Y-%m-%d %H:%M:%S')
            })

    except telerivet.APIException as e:
        print(f"Error occurred: {e}")
        time.sleep(delay)

    return messages

API_KEY = "w_n6I_mg7AINI7n2i3ejMHShsCPYVfevEvqo"
KEN_PROJECT_ID = "PJfea0e3ae2d40f54d"

In [ ]:
def error_log_occurrence(data: list) -> dict:
    """
    Analyze error logs and return occurrence count of each error type.
    """
    try:
        if not data:
            print("Empty array passed")
            return {}
        error_log: dict = {}
        for error in data:
            error_message = error["error log"]
            error_log[error_message] = error_log.get(error_message, 0) + 1
        return error_log
    except Exception as e:
        print(f"Error occurred: {e}")
        return{}

failed_messages = get_failed_messages(API_KEY, KEN_PROJECT_ID)
error_counts = error_log_occurrence(failed_messages)
print(error_counts)